### Starting Spark Session

In [1]:
from pyspark.ml.recommendation import ALS
from pyspark.sql import SparkSession
from tqdm import tqdm

# Create a SparkSession
#spark = SparkSession.builder.getOrCreate()
spark = SparkSession.builder \
    .config('spark.driver.memory', '24g') \
    .getOrCreate()

23/01/15 13:11:00 WARN Utils: Your hostname, darth-Vig800S resolves to a loopback address: 127.0.1.1; using 192.168.100.39 instead (on interface eno1)
23/01/15 13:11:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/15 13:11:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Importing ALS model

In [8]:
from pyspark.ml.recommendation import ALSModel

model_path = "data/als_model1"
#model = ALS.load(model_path)
model = ALSModel.load(model_path)

In [9]:
model

ALSModel: uid=ALS_9f8e159e3d18, rank=10

### Getting Recommendations for Test set parquet by parquet

In [ ]:
# Initialize an empty list to store the DataFrames
df_list = []
x=0
# Iterate through the list of Parquet files
for file in tqdm(file_list):
    # Read the Parquet file into a Spark DataFrame
    df = spark.read.parquet(file)
    userRecs = model.recommendForUserSubset(df, 20)
    userRecs.write.parquet("data/recommendations.userRecs_{}.parquet".format(x))
    x=x+1
    print('file processed',x)

  6%|██▎                                     | 1/17 [24:14<6:27:56, 1454.77s/it]

file processed 1


 12%|████▋                                   | 2/17 [45:59<5:41:40, 1366.69s/it]

file processed 2


 18%|██████▋                               | 3/17 [1:07:33<5:11:03, 1333.13s/it]

...

file processed 9


 59%|█████████████████████▊               | 10/17 [3:34:11<2:27:04, 1260.61s/it]

file processed 10


 65%|███████████████████████▉             | 11/17 [3:55:05<2:05:51, 1258.59s/it]

file processed 11


Code was stopped due to power outage. Started again with the ongoing index

In [10]:
# Initialize an empty list to store the DataFrames
df_list = []
x=12
# Iterate through the list of Parquet files
for file in tqdm(file_list[11:]):
    # Read the Parquet file into a Spark DataFrame
    df = spark.read.parquet(file)
    userRecs = model.recommendForUserSubset(df, 20)
    userRecs.write.parquet("data/recommendations.userRecs_{}.parquet".format(x))
    x=x+1
    print('file processed',x)

 17%|██████▊                                  | 1/6 [22:59<1:54:56, 1379.26s/it]

file processed 13


 33%|█████████████▋                           | 2/6 [45:36<1:31:05, 1366.27s/it]

file processed 14


 50%|█████████████████████▌                     | 3/6 [59:54<56:43, 1134.36s/it]

...

file processed 16


 83%|██████████████████████████████████▏      | 5/6 [1:44:55<21:13, 1273.33s/it]

file processed 17


100%|█████████████████████████████████████████| 6/6 [2:06:08<00:00, 1261.34s/it]

file processed 18


### Getting all parquets containing recommendations and joining them into single pyspark dataframe

In [24]:
import os

# Get the list of files in the data directory
file_list = [f for f in os.listdir('data') if f.endswith('.parquet')]

# Prepend the directory path to each file name
file_list = [os.path.join('data', f) for f in file_list]

In [25]:
#file_list = sorted(file_list)
file_list

['data/recommendations.userRecs_13.parquet',
 'data/recommendations.userRecs_7.parquet',
 'data/recommendations.userRecs_15.parquet',
 'data/recommendations.userRecs_9.parquet',
 'data/recommendations.userRecs_17.parquet',
 'data/recommendations.userRecs_5.parquet',
 'data/recommendations.userRecs_3.parquet',
 'data/recommendations.userRecs_0.parquet',
 'data/recommendations.userRecs_4.parquet',
 'data/recommendations.userRecs_14.parquet',
 'data/recommendations.userRecs_2.parquet',
 'data/recommendations.userRecs_6.parquet',
 'data/recommendations.userRecs_1.parquet',
 'data/recommendations.userRecs_10.parquet',
 'data/recommendations.userRecs_16.parquet',
 'data/recommendations.userRecs_12.parquet',
 'data/recommendations.userRecs_8.parquet']

In [26]:
# Initialize an empty list to store the DataFrames
recs_list = []

# Iterate through the list of Parquet files
for file in tqdm(file_list):
    # Read the Parquet file into a Spark DataFrame
    print('reading', file)
    recs = spark.read.parquet(file)

    # Append the DataFrame to the list
    recs_list.append(recs)

# Concatenate the DataFrames in the list
#df = df_list[0].union(df_list[1:])
recs = recs_list[0]

# Iterate through the rest of the DataFrames in the list
for i in tqdm(range(1, len(recs_list))):
    # Union the DataFrame with the next DataFrame in the list
    recs = recs.union(recs_list[i])

    
recs.show()

 12%|█████▏                                      | 2/17 [00:00<00:00, 19.21it/s]

reading data/recommendations.userRecs_13.parquet
reading data/recommendations.userRecs_7.parquet
reading data/recommendations.userRecs_15.parquet
reading data/recommendations.userRecs_9.parquet


 35%|███████████████▌                            | 6/17 [00:00<00:00, 15.59it/s]

reading data/recommendations.userRecs_17.parquet
reading data/recommendations.userRecs_5.parquet
reading data/recommendations.userRecs_3.parquet


 47%|████████████████████▋                       | 8/17 [00:00<00:00, 13.22it/s]

...

reading data/recommendations.userRecs_1.parquet
reading data/recommendations.userRecs_10.parquet
reading data/recommendations.userRecs_16.parquet


 94%|████████████████████████████████████████▍  | 16/17 [00:01<00:00, 12.40it/s]

reading data/recommendations.userRecs_12.parquet
reading data/recommendations.userRecs_8.parquet


100%|███████████████████████████████████████████| 16/16 [00:00<00:00, 95.38it/s]


+--------+--------------------+
| session|     recommendations|
+--------+--------------------+
|13099782|[{1430595, 6.6899...|
|13099783|[{324469, 6.87379...|
|13099791|[{1009155, 7.0895...|
|13099797|[{1430595, 6.8703...|
|13099798|[{483206, 3.11228...|
|13099802|[{1385885, 4.8944...|
|13099804|[{1430595, 5.2117...|
|13099810|[{864780, 4.63304...|
|13099811|[{1476694, 5.1000...|
|13099812|[{1092859, 4.5939...|
|13099817|[{322904, 6.65523...|
|13099827|[{1113468, 4.9162...|
|13099829|[{784456, 2.37662...|
|13099830|[{54800, 4.843239...|
|13099834|[{984451, 5.71908...|
|13099836|[{1113468, 4.6241...|
|13099838|[{710874, 13.2699...|
|13099842|[{260465, 5.95399...|
|13099850|[{434262, 2.52402...|
|13099863|[{1716873, 6.1223...|
+--------+--------------------+
only showing top 20 rows



In [27]:
recs.count()

1671803

### Saving the dataframe containing recommendations

In [28]:
recs.write.parquet("data/recs.parquet")